In [1]:
import os, random, time
import prophet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#from datetime import date
#import codecs

Importing plotly failed. Interactive plots will not work.


In [2]:
f_list = os.listdir('./data/')
f_list.sort()
print(f_list)

['1_P34_판매계획_2019_10_11_12월.csv', '2_P34_판매계획_2020전체.csv', '3_P34_판매계획_2021_01_02_03월.CSV', '4_P34_생산실적_2020.csv', '5_P34_생산계획_210405.csv', '6_P34_재고실적_210405.csv', 'current.csv', 'post.csv', 'previous.csv']


In [3]:
#### 재고실적 ####
tmp_file = pd.read_csv('./data/'+f_list[5], thousands=',')
tmp_title = ['DATE', 'MONTH', 'WW','DAY', 'IN_CELL', 'OUT_CELL', 'INVENTORY_CELL', 'IN_PACK_D', 'OUT_PACK_D', 'INVENTORY_PACK_D']                
Y_inven = tmp_file[tmp_title].copy()
Y_inven['YEAR'] = pd.to_datetime(tmp_file['DATE']).dt.year

In [4]:
#### 판매계획/실적 Data Set ####
def column_merge(t_tmp, t_title):
    ww_unique = pd.Series.unique(t_title.iloc[0, 15:])
    t_tmp1 = t_tmp[t_title.iloc[0, :14]].copy()
    for j in range(0, len(ww_unique)):
        #t_tmp[ww_unique[j]] = t_tmp[ww_unique[j]].fillna(0)
        if len(t_tmp[ww_unique[j]].shape) > 1 :
            t_tmp1[ww_unique[j]] = t_tmp[ww_unique[j]].fillna(0).sum(axis=1)
        else:
            t_tmp1[ww_unique[j]] = t_tmp[ww_unique[j]].fillna(0)  
    return t_tmp1

tmp_title0 = pd.read_csv('./data/'+f_list[0], skiprows=0, nrows=1)#, header=None)
tmp_title1 = pd.read_csv('./data/'+f_list[1], skiprows=0, nrows=1)#, header=None)
tmp_title2 = pd.read_csv('./data/'+f_list[2], skiprows=0, nrows=1)#, header=None)
tmp0 = pd.read_csv('./data/'+f_list[0], skiprows=2, thousands=',')
tmp1 = pd.read_csv('./data/'+f_list[1], skiprows=2, thousands=',')
tmp2 = pd.read_csv('./data/'+f_list[2], skiprows=2, thousands=',')
tmp0.columns = tmp_title0.loc[0]
tmp1.columns = tmp_title1.loc[0]
tmp2.columns = tmp_title2.loc[0]

tmp_dta0 = column_merge(tmp0, tmp_title0)
tmp_dta1 = column_merge(tmp1, tmp_title1)
tmp_dta2 = column_merge(tmp2, tmp_title2)

index_ww = []
tmp_dta1_ww = pd.to_datetime(tmp_dta1['Version'].str.slice(start=5, stop=13), format='%Y%m%d').dt.isocalendar().week
for w in range(0, len(tmp_dta1_ww)-8): #35,36):#
    if np.divmod(tmp_dta1_ww[w],10)[0] == 0:
        tmp_index_ww = 'W0'+str(tmp_dta1_ww[w])
    else:
        tmp_index_ww = 'W'+str(tmp_dta1_ww[w])
    previous_ww = list(map(lambda x : 'W0'+str(x) if np.divmod(x, 10)[0] == 0 else 'W'+str(x), list(range(tmp_dta1_ww[w]-9, tmp_dta1_ww[w]-1))))
    #print(previous_ww)
    tmp_x_trend = tmp_dta1[previous_ww].iloc[w,:]
    tmp_x_var = tmp_dta1[tmp_index_ww].iloc[w:(w+8)]
    tmp_x = np.concatenate((tmp_x_trend.values, tmp_x_var.values))
    if w == 0:
        #x_trend = tmp_x_trend
        #x_var = tmp_x_var
        tmp_x_sales_plan = tmp_x
    else :
        #x_trend = np.vstack((x_trend, tmp_x_trend))
        #x_var = np.vstack((x_var, tmp_x_var))
        tmp_x_sales_plan = np.vstack((tmp_x_sales_plan,tmp_x))
    index_ww.append(tmp_index_ww)
    
#x_trend_title = list(map(lambda x : 'Xt_'+str(x), list(range(8,0,-1))))
#x_var_title = list(map(lambda x : 'Xv_'+str(x), list(range(1,9))))
X_sales_title = list(map(lambda x : 'Sales_Trend_'+str(x), list(range(8,0,-1)))) + list(map(lambda x : 'Sales_Var_'+str(x), list(range(1,9))))
X_sales_plan = pd.DataFrame(tmp_x_sales_plan)
X_sales_plan.columns = X_sales_title
X_sales_plan['WW'] = index_ww

In [6]:
X_sales_plan

,Sales_Trend_8,Sales_Trend_7,Sales_Trend_6,Sales_Trend_5,Sales_Trend_4,Sales_Trend_3,Sales_Trend_2,Sales_Trend_1,Sales_Var_1,Sales_Var_2,Sales_Var_3,Sales_Var_4,Sales_Var_5,Sales_Var_6,Sales_Var_7,Sales_Var_8,WW
0,901.0,1176.0,1176.0,1323.0,1342.0,1176.0,819.0,903.0,840.0,840.0,420.0,420.0,504.0,252.0,0.0,0.0,W53
1,1092.0,901.0,1176.0,1176.0,1323.0,1342.0,1176.0,819.0,840.0,168.0,168.0,168.0,420.0,840.0,924.0,924.0,W52
2,924.0,1092.0,901.0,1176.0,1176.0,1323.0,1342.0,1176.0,1155.0,1155.0,1155.0,1155.0,1155.0,1239.0,1092.0,819.0,W51
3,1092.0,924.0,1092.0,901.0,1176.0,1176.0,1323.0,1342.0,1407.0,1176.0,1176.0,1260.0,1176.0,1092.0,1260.0,1260.0,W50
4,1008.0,1092.0,924.0,1092.0,901.0,1176.0,1176.0,1323.0,1155.0,1155.0,1155.0,1155.0,1260.0,1239.0,1323.0,1323.0,W49
5,1090.0,1008.0,1092.0,924.0,1092.0,901.0,1176.0,1176.0,1176.0,1155.0,1239.0,1239.0,1239.0,1176.0,1176.0,1176.0,W48
6,1239.0,1090.0,1008.0,1092.0,924.0,1092.0,901.0,1176.0,1176.0,1176.0,1176.0,1176.0,1323.0,1323.0,1323.0,903.0,W47
7,1071.0,1239.0,1090.0,1008.0,1092.0,924.0,1092.0,901.0,1092.0,1008.0,1008.0,756.0,672.0,672.0,840.0,840.0,W46
8,1512.0,1071.0,1239.0,1090.0,1008.0,1092.0,924.0,1092.0,1260.0,1260.0,1260.0,1260.0,1260.0,1260.0,1260.0,1260.0,W45
9,924.0,1512.0,1071.0,1239.0,1090.0,1008.0,1092.0,924.0,735.0,735.0,735.0,819.0,735.0,735.0,735.0,735.0,W44


In [163]:
#### 생산실적 ####
tmp_file = pd.read_csv('./data/'+f_list[3], encoding='euc-kr', thousands=',')
tmp_title = ['Prod Type', 'Material', 'Nick Name', 'Basic Date', 'Plan Date', 'Plan Qty', 'Result Qty']
tmp_p_real1 = tmp_file[tmp_title].copy()
tmp_p_real1['YEAR'] = pd.to_datetime(tmp_file['Plan Date']).dt.isocalendar().year
tmp_p_real1['WW'] = pd.to_datetime(tmp_file['Plan Date']).dt.isocalendar().week
X_product_actual = tmp_p_real1[(tmp_p_real1['Prod Type']=='MASS') & (tmp_p_real1['YEAR']==2020)] 
print(X_product_actual)

      Prod Type          Material  Nick Name  Basic Date   Plan Date  \
545        MASS  APPAG12V0-A3-C08  PAG-12V-S  2020-01-05  2020-01-02   
546        MASS  APPAG12V0-A3-C08  PAG-12V-S  2020-01-05  2020-01-03   
547        MASS  APPAG12V0-A3-C08  PAG-12V-S  2020-01-05  2020-01-04   
548        MASS  APPAG12V0-A3-C08  PAG-12V-S  2020-01-05  2020-01-05   
549        MASS  APPAG12V0-A3-C08  PAG-12V-S  2020-01-12  2020-01-07   
...         ...               ...        ...         ...         ...   
12903      MASS  APPSAPHEV-A1-D03   PSA-PHEV  2020-12-20  2020-12-18   
12904      MASS  APPSAPHEV-A1-D03   PSA-PHEV  2020-12-27  2020-12-21   
12905      MASS  APPSAPHEV-A1-D03   PSA-PHEV  2020-12-27  2020-12-22   
12906      MASS  APPSAPHEV-A1-D03   PSA-PHEV  2020-12-27  2020-12-23   
12907      MASS  APPSAPHEV-A1-D03   PSA-PHEV  2021-01-03  2020-12-28   

       Plan Qty  Result Qty  YEAR  WW  
545         480         504  2020   1  
546         560         584  2020   1  
547         560

In [160]:
#### 생산실적 ####
tmp_file = pd.read_csv('./data/'+f_list[3], encoding='euc-kr', thousands=',')
tmp_title = ['Prod Type', 'Material', 'Nick Name', 'Basic Date', 'Plan Date', 'Plan Qty', 'Result Qty']
tmp_p_real1 = tmp_file[tmp_title].copy()
tmp_p_real1['YEAR'] = pd.to_datetime(tmp_file['Plan Date']).dt.isocalendar().year
tmp_p_real1['WW'] = pd.to_datetime(tmp_file['Plan Date']).dt.isocalendar().week
X_product_actual = tmp_p_real1[(tmp_p_real1['Prod Type']=='MASS') & (tmp_p_real1['YEAR']==2020)] 

cnt = 0
index_TY_ww = []
for t in range(53,9, -1):
    TW=t;TY=2020
    r_QTY_sum = list()
    for AW in range(TW-1, TW-9, -1):
        r_QTY_sum.append(X_product_actual[(X_product_actual['YEAR']==TY) & (X_product_actual['WW']==AW)]['Result Qty'].sum())
    if cnt == 0:
        tmp_X_product_actual = r_QTY_sum
    else:
        tmp_X_product_actual = np.vstack((tmp_X_product_actual, r_QTY_sum))
    cnt+=1
    if np.divmod(t, 10)[0] == 0:
        index_TY_ww.append('W0'+str(t))
    else:
        index_TY_ww.append('W'+str(t))        

tmp_X_product_actual = pd.DataFrame(tmp_X_product_actual)
tmp_X_product_actual_title = list(map(lambda x : 'Product_Trend_'+str(x), list(range(1,9))))
tmp_X_product_actual.columns = tmp_X_product_actual_title
tmp_X_product_actual['WW'] = index_TY_ww

#### 생산계획 ####
tmp_file = pd.read_csv('./data/'+f_list[4], thousands=',')
tmp_title = ['PLAN_START_YYYYMMDD','YYYYMMDD', 'QTY']
X_product_plan = tmp_file[tmp_title].copy() 
X_product_plan['PLAN_WW'] = pd.to_datetime(tmp_file['PLAN_START_YYYYMMDD'], format='%Y%m%d').dt.isocalendar().week
X_product_plan['PLAN_YEAR'] = pd.to_datetime(tmp_file['PLAN_START_YYYYMMDD'], format='%Y%m%d').dt.isocalendar().year
X_product_plan['TARGET_WW'] = pd.to_datetime(tmp_file['YYYYMMDD'], format='%Y%m%d').dt.isocalendar().week
X_product_plan['TARGET_YEAR'] = pd.to_datetime(tmp_file['YYYYMMDD'], format='%Y%m%d').dt.isocalendar().year

time_key = X_product_plan.drop_duplicates(['PLAN_YEAR', 'PLAN_WW','TARGET_YEAR','TARGET_WW'], keep='first')

cnt = 0
index_TY_ww = []
for t in range(53, 9, -1):
    #PW, PY, _, TY = time_key.iloc[t][3:]
    QTY_sum = list()
    for PW in range(t, t-8, -1):        
        TW=t; TY=2020 ; PY=2020
        QTY_sum.append(X_product_plan[(X_product_plan['PLAN_YEAR']==PY) & (X_product_plan['PLAN_WW']==PW) & (X_product_plan['TARGET_YEAR']==TY) & (X_product_plan['TARGET_WW']==TW)]['QTY'].sum())

    if cnt== 0:
        tmp_X_product_plan = QTY_sum
    else:
        tmp_X_product_plan = np.vstack((tmp_X_product_plan, QTY_sum))
    cnt+=1
    if np.divmod(t, 10)[0] == 0:
        index_TY_ww.append('W0'+str(t))
    else:
        index_TY_ww.append('W'+str(t))

tmp_X_product_plan = pd.DataFrame(tmp_X_product_plan)
tmp_X_product_plan_title = list(map(lambda x : 'Product_Var'+str(x), list(range(1,9))))
tmp_X_product_plan.columns = tmp_X_product_plan_title
tmp_X_product_plan['WW'] = index_TY_ww
X_product_plan = pd.merge(tmp_X_product_plan, tmp_X_product_actual, how='inner', on = 'WW')



In [162]:
X_product_plan #X_sales_plan

,Product_Var1,Product_Var2,Product_Var3,Product_Var4,Product_Var5,Product_Var6,Product_Var7,Product_Var8,WW,Product_Trend_1,Product_Trend_2,Product_Trend_3,Product_Trend_4,Product_Trend_5,Product_Trend_6,Product_Trend_7,Product_Trend_8
0,1060,1110,1110,1210,1210,1210,1210,1210,W53,13905,30134,31232,23225,20594,18726,23981,20484
1,1110,1110,1100,1210,1100,1100,1100,1100,W52,30134,31232,23225,20594,18726,23981,20484,19919
2,950,440,440,440,440,440,440,440,W51,31232,23225,20594,18726,23981,20484,19919,28506
3,847,880,880,880,660,660,880,770,W50,23225,20594,18726,23981,20484,19919,28506,19320
4,1320,1210,1210,1210,1100,880,1100,1210,W49,20594,18726,23981,20484,19919,28506,19320,19677
5,1210,1210,1210,1210,1210,1210,1210,1210,W48,18726,23981,20484,19919,28506,19320,19677,17274
6,660,880,1210,1210,1210,1210,1210,990,W47,23981,20484,19919,28506,19320,19677,17274,19710
7,1100,1100,1100,1100,1100,1100,880,880,W46,20484,19919,28506,19320,19677,17274,19710,22629
8,1100,1100,1100,1100,1100,1100,1100,1100,W45,19919,28506,19320,19677,17274,19710,22629,21428
9,1210,1100,880,880,880,880,1100,1100,W44,28506,19320,19677,17274,19710,22629,21428,19078


In [157]:
tmp_x

,Product_Var1,Product_Var2,Product_Var3,Product_Var4,Product_Var5,Product_Var6,Product_Var7,Product_Var8,WW,Product_Trend_1,Product_Trend_2,Product_Trend_3,Product_Trend_4,Product_Trend_5,Product_Trend_6,Product_Trend_7,Product_Trend_8
0,1060,1110,1110,1210,1210,1210,1210,1210,W53,13905,30134,31232,23225,20594,18726,23981,20484
1,1110,1110,1100,1210,1100,1100,1100,1100,W52,30134,31232,23225,20594,18726,23981,20484,19919
2,950,440,440,440,440,440,440,440,W51,31232,23225,20594,18726,23981,20484,19919,28506
3,847,880,880,880,660,660,880,770,W50,23225,20594,18726,23981,20484,19919,28506,19320
4,1320,1210,1210,1210,1100,880,1100,1210,W49,20594,18726,23981,20484,19919,28506,19320,19677
5,1210,1210,1210,1210,1210,1210,1210,1210,W48,18726,23981,20484,19919,28506,19320,19677,17274
6,660,880,1210,1210,1210,1210,1210,990,W47,23981,20484,19919,28506,19320,19677,17274,19710
7,1100,1100,1100,1100,1100,1100,880,880,W46,20484,19919,28506,19320,19677,17274,19710,22629
8,1100,1100,1100,1100,1100,1100,1100,1100,W45,19919,28506,19320,19677,17274,19710,22629,21428
9,1210,1100,880,880,880,880,1100,1100,W44,28506,19320,19677,17274,19710,22629,21428,19078
